<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-11/blob/main/Primer%20modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer modelo

# Preparación del entorno de trabajo

Lo primero que vamos a hacer es preparar el entorno de trabajo. Para ello, descargaremos e importaremos las librerías necesarias:

Instalamos las librerías necesarias:

In [1]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 204kB 5.0MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 


Las importamos:

In [2]:
from fastai.vision.all import *
import os
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
import shutil

Descargamos las imágenes:

In [3]:
!wget https://www.dropbox.com/s/yqciiv18j73y578/riojamemoria.zip?dl=1 -O datos.zip -q
!unzip datos.zip

Archive:  datos.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of datos.zip or
        datos.zip.zip, and cannot find datos.zip.ZIP, period.


Una vez tenemos las imágenes, debemos descargar las etiquetas. Cada etiqueta se relaciona con su imagen correspondiente por un ID (el nombre de la imagen). Estas etiquetas son necesarias para poder entrenar nuestro modelo.

In [6]:
!wget https://www.dropbox.com/s/acezkcl7d1wfevz/LaRiojaEnLaMemoria.csv?dl=1 -O data.csv -q

Importamos los datos del CSV a un dataframe (usando pandas) para poder tratarlos.

In [17]:
df = pd.read_csv("data.csv")
df.head()

,ID_NUM,IDENT,TITULO,TITULAR,DESCRIPCION,PALABRAS_CLAVE,ANNO,LUGAR,LOCALIDAD,PROVINCIA,PAIS,PROVEEDOR
0,80,LRM-2008/00014-Imagen,Foto con los abuelos,Fotografía de los abuelos con la nieta de dos años en Hormilleja,NaN,NaN,1961,NaN,Hormilleja,NaN,NaN,NaN
1,81,LRM-2008/00015-Imagen,Madre e hijos en el Espolón,Madre e hijos en el Espolón,NaN,NaN,1961,NaN,Logroño,NaN,NaN,NaN
2,82,LRM-2008/00016-Imagen,SrTraspaderne,Severiano Traspaderne con su furgoneta Citroen 2CV de reparto de piensos,NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN
3,83,LRM-2008/00017-Imagen,Invierno en moto,Sr Aragón y Sr García pertrechados para soportar el invierno en su motol (Rieju 125),NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN
4,84,LRM-2008/00018-Imagen,Empujando el microcoche,Empujando un BMW Isetta,NaN,NaN,1962,NaN,Logroño,NaN,NaN,NaN


Ahora, creamos una columna adicional que nos permita clasificar las imágenes según su decada:

In [21]:
años = df['ANNO'].values
decadas = []
for año in años:
  decadas.append(str(año)[:3] + '0')
df['DECADA'] = decadas
df.head()

,ID_NUM,IDENT,TITULO,TITULAR,DESCRIPCION,PALABRAS_CLAVE,ANNO,LUGAR,LOCALIDAD,PROVINCIA,PAIS,PROVEEDOR,DECADA
0,80,LRM-2008/00014-Imagen,Foto con los abuelos,Fotografía de los abuelos con la nieta de dos años en Hormilleja,NaN,NaN,1961,NaN,Hormilleja,NaN,NaN,NaN,1960
1,81,LRM-2008/00015-Imagen,Madre e hijos en el Espolón,Madre e hijos en el Espolón,NaN,NaN,1961,NaN,Logroño,NaN,NaN,NaN,1960
2,82,LRM-2008/00016-Imagen,SrTraspaderne,Severiano Traspaderne con su furgoneta Citroen 2CV de reparto de piensos,NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN,1960
3,83,LRM-2008/00017-Imagen,Invierno en moto,Sr Aragón y Sr García pertrechados para soportar el invierno en su motol (Rieju 125),NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN,1960
4,84,LRM-2008/00018-Imagen,Empujando el microcoche,Empujando un BMW Isetta,NaN,NaN,1962,NaN,Logroño,NaN,NaN,NaN,1960


Finalmente, etiquetamos las imágenes a partir de la información de los metadatos.

Para ello, vamos a obtener lAs diferentes décadas en las que están tomadas las imágenes:

In [33]:
decades = set(df['DECADA'])
print(decades)

{'1910', '1950', '1900', '1860', '1930', '1980', '1940', '1990', '1970', '1880', '1890', '1920', '1870', '1960'}


Creamos las carpetas contenedoras. El nombre de la carpeta es el año en que la foto está tomada.

In [34]:
initialPath = Path('1')
processedPath = Path('data')
for decade in decades:
  (processedPath/str(decade)).mkdir(parents=True,exist_ok=True)

Solo falta meter en cada carpeta las imágenes correspondientes. Para ello, definimos dos funciones:
- getIdFromPath: devuelve el id de la imagen según su path.
- getImageDecadeById(): devuelve la década de la imagen según su id.

In [35]:
def getIdFromPath(path):
  pathAsString = str(path)
  pointIndex = pathAsString.index(".")
  pathAsString = pathAsString[2:pointIndex]
  return pathAsString

In [36]:
def getImageDecadeById(id):
  try:
    return df[df['ID_NUM'] == int(id)]['DECADA'].values[0]
  except:
    return -1

Para cada imagen, obtenemos su id, buscamos su año en la tabla y la movemos a la carpeta del año a la que pertenece.

Si la imagen no tuviere un año asignado, quedaría eliminada (estas imagenes no nos servieren para entrenar el modelo).

In [ ]:
images = get_image_files(initialPath)

for image in images:
  imageId = getIdFromPath(image)
  imageYear = getImageYearById(imageId)
  if imageYear != -1:
    shutil.move(str(image),processedpath/(str(imageYear)+'/'+image.name))
  else:
    print("No data for image " + str(id))
    os.remove(image)

Como se puede observar, hay algunas imágenes que no están recogidas en la tabla, por lo que no las podremos usar.

Finalmente, eliminamos las carpetas y archivos que ya no vamos a utilizar, así como las imágenes inexistentes en la tabla.

In [ ]:
shutil.rmtree('1')
os.remove('data.csv')
os.remove('datos.zip')

## Separación en entrenamiento y test

In [ ]:
TODO